In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import *
import random
import time
from scipy.fft import *

In [3]:
# Load the file into a pandas DataFrame
df =pd.read_csv('/content/MW.csv')
df.columns = ['id', 'event_id', 'device', 'channel', 'code', 'size', 'data']

#  convert each comma-separated string in the "data" column of a DataFrame df into a list of floats.(use apply and lambda)
df['data'] = df['data'].str.split(',')
df['data'] = df['data'].apply(lambda x: x if isinstance(x, list) else [])



# Display the first few rows of the DataFrame
print (df.head())

   id  event_id device channel  code  size  \
0   0         0     MW     FP1     0  1017   
1   1         1     MW     FP1     1   889   
2   2         2     MW     FP1     4  1017   
3   3         3     MW     FP1     1  1017   
4   4         4     MW     FP1     8   952   

                                                data  
0  [38, 48, 51, 44, 48, 56, 56, 41, 20, -3, -9, -...  
1  [83, 74, 65, 65, 66, 55, 43, 25, 18, 20, 26, 3...  
2  [19, 10, -2, -9, -5, 3, 8, 7, 8, 2, -10, -21, ...  
3  [17, 29, 36, 26, 21, 17, 17, 13, 17, 21, 25, 2...  
4  [77, 74, 69, 70, 76, 75, 76, 75, 67, 66, 76, 9...  


In [4]:
# Function to resample an array to the target length
def resample_array(array, target_length):
    # Create an array of indices for the input array
    input_indices = np.linspace(0, len(array)-1, len(array))

    # Create an array of indices for the resampled array
    resampled_indices =np.linspace(0,len(array)-1,target_length)

    # Create a linear interpolation function based on the input array
    interpolator =interp1d(input_indices,array,'linear')

    # Use the interpolator to create the resampled array
    resampled_array =interpolator(resampled_indices)

    return resampled_array.tolist()

In [6]:
median_length=int(df["size"].median())
# Resample all the data arrays to the median length(use lambda function and apply)
df['resampled_data'] = df.apply(lambda row: resample_array(row['data'],median_length), axis=1)

# Check the length of the resampled arrays
df["resampled_data_length"] = df["resampled_data"].apply(len)

# Display the first few rows of the updated DataFrame
print(df.head())

   id  event_id device channel  code  size  \
0   0         0     MW     FP1     0  1017   
1   1         1     MW     FP1     1   889   
2   2         2     MW     FP1     4  1017   
3   3         3     MW     FP1     1  1017   
4   4         4     MW     FP1     8   952   

                                                data  \
0  [38, 48, 51, 44, 48, 56, 56, 41, 20, -3, -9, -...   
1  [83, 74, 65, 65, 66, 55, 43, 25, 18, 20, 26, 3...   
2  [19, 10, -2, -9, -5, 3, 8, 7, 8, 2, -10, -21, ...   
3  [17, 29, 36, 26, 21, 17, 17, 13, 17, 21, 25, 2...   
4  [77, 74, 69, 70, 76, 75, 76, 75, 67, 66, 76, 9...   

                                      resampled_data  resampled_data_length  
0  [38.0, 48.20168067226891, 50.05882352941177, 4...                    953  
1  [83.0, 74.60504201680672, 66.21008403361344, 6...                    953  
2  [19.0, 9.193277310924369, -2.941176470588235, ...                    953  
3  [17.0, 29.470588235294116, 34.65546218487395, ...                    95

In [5]:
data_array = np.array(df["resampled_data"].tolist())
codes = df['code'].tolist()

In [6]:
srate = 220
def time_frequency(data, cmwX, nKern):
    ''''
    Function to calculate time-frequency representation of multichannel data.

    Parameters:
    data : ndarray
        The EEG data, array of shape (channels, time).
    cmwX : ndarray
        The Fourier coefficients of the complex Morlet wavelets, array of shape (frequencies, nConv).
    nKern : int
        The length of the wavelet kernel.
    channel_labels : list, optional
        The labels of the EEG channels. Must be the same length as the number of channels in the data.
        If not provided, no channel labels will be used.

    Returns:
    tf : ndarray
        The time-frequency representation of the data, array of shape (frequencies, time).
        This is the average power across all channels.
    '''

    # set up convolution parameters
    nData   = data.shape[1]
    nConv   = nData + nKern - 1
    halfwav = (nKern-1)//2

    # initialize time-frequency output matrix
    tf = np.zeros((data.shape[0], cmwX.shape[0], data.shape[1])) # channels X frequency X times

    # loop over channels
    for chani in range(data.shape[0]):

        # compute Fourier coefficients of EEG data
        eegX = fft(data[chani, :] , nConv)

        # perform convolution and extract power (vectorized across frequencies)
        as_ = ifft(cmwX * eegX[None, :], axis=1)
        as_ = as_[:, halfwav: -halfwav]
        tf[chani, :, :] = np.abs(as_) ** 2

    return tf


def get_cmwX(nData, freqrange=[1,40], numfrex=42):
    '''
    Function to calculate the Fourier coefficients of complex Morlet wavelets.

    Parameters:
    nData : int
        The number of data points.
    freqrange : list, optional
        The range of frequencies to consider. Defaults to [1,40].
    numfrex : int, optional
        The number of frequencies between the lowest and highest frequency. Defaults to 42.

    Returns:
    cmwX : ndarray
        The Fourier coefficients of the complex Morlet wavelets, array of shape (frequencies, nConv).
    nKern : int
        The length of the wavelet kernel.
    frex : ndarray
        The array of frequencies.
    '''
    pi = np.pi
    wavtime = np.arange(-2,2-1/srate,1/srate)
    nKern = len(wavtime)
    nConv = nData + nKern - 1
    frex = np.linspace(freqrange[0],freqrange[1],numfrex)
   # create complex morlet wavelets array
    cmwX = np.zeros((numfrex, nConv), dtype=complex)

    # number of cycles
    numcyc = np.linspace(3,15,numfrex);
    for fi in range(numfrex):
        # create time-domain wavelet
        s = numcyc[fi] / (2*pi*frex[fi])
        twoSsquared = (2*s) ** 2
        cmw = np.exp(2*1j*pi*frex[fi]*wavtime) * np.exp( (-wavtime**2) / twoSsquared )


        # compute fourier coefficients of wavelet and normalize
        cmwX[fi, :] = fft(cmw, nConv)
        cmwX[fi, :] = cmwX[fi, :] / max(cmwX[fi, :])

    return cmwX, nKern, frex

In [ ]:
starting_freq = 1
end_freq = 6
num_frequencies = 10

#  Creates an array of time values starting from 0 to 2 seconds, with a total number of elements defined by median_length.
times = np.linspace(0,2,median_length)

nData = data_array.shape[1]
cmwX, nKern, frex = get_cmwX(nData, freqrange=[starting_freq, end_freq], numfrex=num_frequencies)
tf = time_frequency(data_array, cmwX, nKern)

fig, axs = plt.subplots(3, 3, figsize=(15, 10))

for i,ax in enumerate(axs.flat):
  x = random.randint(0, tf.shape[0])
  contour = ax.contourf(times, frex, tf[x,:,:], 40, cmap='jet')
  ax.set_xlabel('Time (s)')
  ax.set_ylabel('Frequencies (Hz)')
  ax.set_title(f"Time Frequency Plot for {'non-digit' if codes[x] == -1 else 'digit'}")

fig.tight_layout()